<a href="https://colab.research.google.com/github/Nikhil-Kadapala/NeuralNets/blob/main/standardNeuralNets/stdCNN_LIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Project CS852 - Foundations of Neural Networks (FALL 2024)





Devin Borchard and Nikhil Kadapala

Department of Computer Science, University of New Hampshire

ERASER datasets: https://www.eraserbenchmark.com/

ERASER paper: https://arxiv.org/pdf/1911.03429

LIME paper: https://arxiv.org/pdf/1602.04938

# Notebook setup and PyTorch Installation

In [2]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# uncomment one of these versions (depending on whether you are on a computer with a CPU or not)

# GPU version
# !conda install --yes --prefix {sys.prefix} pytorch torchvision cudatoolkit=10.2 -c pytorch

# Just CPU
# !conda install --yes --prefix {sys.prefix} pytorch torchvision cpuonly -c pytorch

# install `Einops` for einstein-style tensor manipulation in pytorch
# Also see https://github.com/arogozhnikov/einops
# !conda install --yes --prefix {sys.prefix} einops  -c conda-forge


In [3]:
# torch test
import torch
x = torch.rand(5, 3)
print(x)

print("GPU/CUDA available? ", torch.cuda.is_available())

print("Torch version", torch.__version__)

tensor([[0.1309, 0.1836, 0.9098],
        [0.5933, 0.8962, 0.0454],
        [0.0172, 0.8703, 0.0298],
        [0.9056, 0.4607, 0.0746],
        [0.5066, 0.6399, 0.1183]])
GPU/CUDA available?  True
Torch version 2.5.1+cu121


# **Extracting Traning, Validation, and Test Data**
# Parse the data files to extract the reviews, classifications and annotations for each split.

There are three files:
- train.jsonl: containts 1600 training examples
- val.jsonl: contains 200 validation examples
- test.json: contains 199 test examples

Each example includes:
- annotation_id: a unique id for an example of the form negR_000 for negative examples and posR_000 for positive examples.
- evidences: a list of rationales(specific parts of the review) given by humans that most influenced their classification decision.
- classification: the class of the example

The annotation_id of each example is the name of the file for the input text data
    

In [4]:
import json

def parse_data(file_path):
    data = []                                               # Initialize an empty list to store the dictionaries

    with open(file_path, 'r') as file:                      # Open the .jsonl file and read it line by line
        for line in file:
            annotation = json.loads(line)                   # Parse each line as JSON and append it to the list
            id = annotation["annotation_id"]
            annotation["classification"] = 1 if annotation['classification'] == "POS" else 0

            with open(f"./movies/docs/{id}", 'r') as file:  # open the file named by annotation_id to extract the review text
                content = file.read()
                annotation['content'] = content.replace('\n', ' ')
                data.append(annotation)
    return data

# Specify the path to your JSON file
train_file_path = './movies/train.jsonl'
val_file_path = './movies/val.jsonl'
test_file_path = './movies/test.jsonl'

train_data = parse_data(train_file_path)
validation_data = parse_data(val_file_path)
test_data = parse_data(test_file_path)

# Functions to extract reviews, classifications, and annotations
  Define a function

  i) to retrieve an example and print the relevant information.

  ii) to retrieve the content of the example review text

  iii) to retrieve the classifications of the examples
  
  iv) to retrieve the annotations provided to support the classifications

In [5]:
def print_example(data, index, print_content=True, print_classification=True, print_rationales=True ):
    print(f'Retrieving Training Example [{index}].................\n')
    item = data[index]
    classification = item['classification']
    evidences = item['evidences']
    content = item['content']
    if print_content: print(f'Review content:\n{content}\n')
    if print_classification: print('----------------------------',
                                   '\n| Sentiment class:',
                                   classification,
                                   ("- NEG" if not classification else "- POS"),
                                   '|', '\n----------------------------')
    if print_rationales:
        print('\nHuman rationales / Supporting Evidence:')
        for evidence in evidences:
            print('     - ', evidence[0]['text'])

def get_content(data, index):
    item = data[index]
    content = item['content']
    return content

def get_classes(data, index):
    item = data[index]
    classification = item['classification']
    return classification

def get_annotations(data, index):
    item = data[index]
    content = item['evidences']
    annotations = [evidence[0]['text'] for evidence in content]
    return annotations

train_size = len(train_data)
val_size = len(validation_data)
test_size = len(test_data)

print(f'Dataset split: {train_size} training examples')
print(f'               {val_size} validation examples')
print(f'               {test_size} test examples\n')

print_example(train_data, 506)

Dataset split: 1600 training examples
               200 validation examples
               199 test examples

Retrieving Training Example [506].................

Review content:
this film is extraordinarily horrendous and i 'm not going to waste any more words on it .

---------------------------- 
| Sentiment class: 0 - NEG | 
----------------------------

Human rationales / Supporting Evidence:
     -  extraordinarily horrendous


# Extraction of the rationales from the evidences metadata of each human annotation of reviews.

Each annotation of the review is not the highlighted text/rationale itself but also contains metadata of the text. Use the function defined in the above cell to extract just the text and replace the evidences dictionary of the training, validation, and test datasets.

In [6]:
for i in range(len(train_data)):
    train_data[i]['evidences'] = get_annotations(train_data, i)

for i in range(len(validation_data)):
    validation_data[i]['evidences'] = get_annotations(validation_data, i)

for i in range(len(test_data)):
    test_data[i]['evidences'] = get_annotations(test_data, i)

print(train_data[506]['evidences'])

['extraordinarily horrendous']


# Pre-trianed GloVe Embeddings of Training Examples
Download the pretrained GloVe Embeddings of desired dimensions using gensim downlader.

Save downloaded embeddings to a local file to avoid re-downloading when the kernel or notebook is restarted.

In [7]:
"""
    Install gensim, to use word2vec word embeddings
    Install gensim (for pre-trained word embeddings)
    #!conda install --yes --prefix {sys.prefix} gensim
"""
#import gensim
#import gensim.downloader

"""
    ONLY if you get an error after `import gensim`: update your smart_open liberary
    #!conda install --yes --prefix {sys.prefix} smart_open
    restart your notebook
    see if `import gensim` works now
"""
#wv = gensim.downloader.load("glove-wiki-gigaword-50")

#import pickle

#with open("glove_embeddings.pkl", "wb") as f:
    #pickle.dump(wv, f)


'\n    ONLY if you get an error after `import gensim`: update your smart_open liberary\n    #!conda install --yes --prefix {sys.prefix} smart_open\n    restart your notebook\n    see if `import gensim` works now\n'

In [8]:
import pickle

with open("glove_embeddings.pkl", "rb") as f:
    wv = pickle.load(f)

# lookup the word vector for a word "india"
wv['india']

array([-0.20356 , -0.8707  , -0.19172 ,  0.73862 ,  0.18494 ,  0.14926 ,
        0.48079 , -0.21633 ,  0.72753 , -0.36912 ,  0.13397 , -0.1143  ,
       -0.18075 , -0.64683 , -0.18484 ,  0.83575 ,  0.48179 ,  0.76026 ,
       -0.50381 ,  0.80743 ,  1.2195  ,  0.3459  ,  0.22185 ,  0.31335 ,
        1.2066  , -1.8441  ,  0.14064 , -0.99715 , -1.1402  ,  0.32342 ,
        3.2128  ,  0.42708 ,  0.19504 ,  0.80113 ,  0.38555 , -0.12568 ,
       -0.26533 ,  0.055264, -1.1557  ,  0.16836 , -0.82228 ,  0.20394 ,
        0.089235, -0.60125 , -0.032878,  1.3735  , -0.51661 ,  0.29611 ,
        0.23951 , -1.3801  ], dtype=float32)

In [9]:
# downsampled embedding and zero vector for unknown words
# note the following code assums the the word embedding dimensions are dividible by 5

import einops # type: ignore
import numpy as np
from typing import List
import types

def glove_embed(word:str, target_dim)->np.array:
    '''Looks up word in embedding (downsampled to five dimensions), pads with beginning of embedding.
       Returns zero vector for unknown words.
    '''
    # these parameters work for 50-dim glove embeddings (adjust for other embeddings)
    sampled_dim = 5
    sample_batches = 10

    empty_vec=np.zeros(target_dim)
    if word in wv:
        w2v = wv[word] # lookup 50 dim vector
        a=einops.reduce(w2v,'(d seg)-> d', "sum", seg=sample_batches)  # downsample
        b=w2v[0:target_dim-sampled_dim]
        return np.hstack([a,b])
    else:
        return empty_vec

def glove_embed_sequences(sequence, target_dim):

    if isinstance(sequence, list):
        if len(sequence) == 0:
            empty_seq = np.zeros(target_dim)
            gloveTensor =  torch.tensor(empty_seq, dtype=torch.float)
        else:
            tokens = ",".join(sequence)
            words = tokens.split()
            gloveTensor = torch.stack([torch.tensor(glove_embed(word, target_dim), dtype=torch.float) for word in words])
    else:
        tokens = sequence.split()
        gloveTensor = torch.stack([torch.tensor(glove_embed(token, target_dim), dtype=torch.float) for token in tokens])

    return gloveTensor

# Extract reviews, classifications, and rationales from the train, validation, and test datasets to convert them to Glove embeddings.

In [10]:
# convert the training dataset to a pandas dataframe
train_df = pd.DataFrame(train_data)
train_df.drop(columns=['query', 'query_type'], inplace=True)
train_df['evidences'] = train_df['evidences'].astype(str)

train_rationales = train_df['evidences']
train_reviews = [get_content(train_data, i) for i in range(train_size)]
train_classes = torch.tensor([get_classes(train_data, i) for i in range(train_size)], dtype=torch.float)

print("Number of reviews in training data:",len(train_reviews))
print("Max seq length of reviews:", np.max([len(review.split()) for review in train_reviews]))
train_df.to_csv('train_data.csv', index=False)
train_df.head()

Number of reviews in training data: 1600
Max seq length of reviews: 2809


,annotation_id,classification,evidences,content
0,negR_000.txt,0,"['mind - fuck movie', 'the sad part is', 'down...","plot : two teen couples go to a church party ,..."
1,negR_001.txt,0,"[""it 's pretty much a sunken ship"", 'sutherlan...",the happy bastard 's quick movie review damn t...
2,negR_002.txt,0,['the characters and acting is nothing spectac...,it is movies like these that make a jaded movi...
3,negR_003.txt,0,"['dead on arrival', 'the characters stink', 's...",""" quest for camelot "" is warner bros . ' first..."
4,negR_004.txt,0,['it is highly derivative and somewhat boring'...,synopsis : a mentally unstable man undergoing ...


In [11]:
# convert the validation dataset to a pandas dataframe
val_df = pd.DataFrame(validation_data)
val_df.drop(columns=['query', 'query_type'], inplace=True)
val_df['evidences'] = val_df['evidences'].astype(str)

val_rationales = val_df['evidences']
val_reviews = [get_content(validation_data, i) for i in range(val_size)]
val_classes = torch.tensor([get_classes(validation_data, i) for i in range(val_size)], dtype=torch.float)

print("Number of reviews in validation data:",len(val_reviews))
print("Max seq length of reviews:", np.max([len(review.split()) for review in val_reviews]))
val_df.to_csv('val_data.csv', index=False)
val_df.head()

Number of reviews in validation data: 200
Max seq length of reviews: 1880


,annotation_id,classification,evidences,content
0,negR_800.txt,0,['definitely the cinematic equivalent of a sle...,there were four movies that earned jamie lee c...
1,negR_801.txt,0,"['overacts his psycho routine', 'deteriorates ...",according to hitchcock and various other filmm...
2,negR_802.txt,0,"['so dull and pedestrian and nonsensical', 'bo...",if you 've been following william fichtner 's ...
3,negR_803.txt,0,"['takes the easy route out', 'most hampered no...",note : some may consider portions of the follo...
4,negR_804.txt,0,"['poor choices', ""it 's downright depressing"",...","for his directoral debut , gary oldman chose a..."


In [12]:
# convert the test dataset to a pandas dataframe
test_df = pd.DataFrame(test_data)
test_df.drop(columns=['docids','query', 'query_type'], inplace=True)
test_df['evidences'] = test_df['evidences'].astype(str)

test_rationales = test_df['evidences']
test_reviews = [get_content(test_data, i) for i in range(test_size)]
test_classes = torch.tensor([get_classes(test_data, i) for i in range(test_size)], dtype=torch.float)

print("Number of reviews in test data:",len(test_reviews))
print("Max seq length of reviews:", np.max([len(review.split()) for review in test_reviews]))
test_df.to_csv('test_data.csv', index=False)
test_df.head()

Number of reviews in test data: 199
Max seq length of reviews: 2122


,annotation_id,classification,evidences,content
0,negR_900.txt,0,['i even giggled'],there may not be a critic alive who harbors as...
1,negR_901.txt,0,['rings'],"renee zellweger stars as sonia , a young jewis..."
2,negR_902.txt,0,"[""there 're so many things to criticize about ...",there 're so many things to criticize about i ...
3,negR_903.txt,0,"[""do n't let this movie fool you into believin...",do n't let this movie fool you into believing ...
4,negR_904.txt,0,"[""is proof that hollywood does n't have a clue...",it 's a good thing most animated sci - fi movi...


Extract validation set from the val.jsonl file and create a dataframe for it similar to the training set and save it to a csv file.

# Convert the reviews & rationales to their corresponding Glove embeddings

In [13]:
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

def create_glove_dict(sequence, wv, set, embed_dim=50):
    """
    Creates a dictionary mapping words in the vocabulary to their GloVe embeddings.
    Words that don't exist are mapped to zero vectors.
    """
    glove_dict = {}
    empty_vec = np.zeros(embed_dim, dtype=np.float64)

    for word in tqdm(sequence, desc=f"Building {set} GloVe dictionary"):
        glove_dict[word] = wv[word] if word in wv else empty_vec

    return glove_dict

def get_w2GloVe(data, glove_dict, set, embed_dim=50, rationale=False):
    """
    Retrieves the GloVe embeddings using the custom-built GloVe dictionary.
    Args:
        data: List of text reviews.
        glove_dict (dict): custom-built GloVe dictionary.
        embed_dim (int): Dimensions of GloVe embeddings.
    Returns:
        torch.Tensor: Padded tensor of GloVe embeddings to maintain uniform length.
    """
    glove_reviews = []

    if rationale:
        for review in tqdm(data, desc=f"Retrieving {set} GloVe Word Embeddings"):
            tokens = ",".join(review)
            words = tokens.split()
            embeddings = [glove_dict.get(word, np.zeros(embed_dim)) for word in words]
            glove_reviews.append(torch.tensor(embeddings, dtype=torch.float))
    else:
        for review in tqdm(data, desc=f"Retrieving {set} GloVe Word Embeddings"):
            words = review.split()
            embeddings = [glove_dict.get(word, np.zeros(embed_dim)) for word in words]
            glove_reviews.append(torch.tensor(embeddings, dtype=torch.float))

    return pad_sequence(glove_reviews, batch_first=True)

In [14]:
print(f"----------------------------------------------------------------------------------------\nProcessing Reviews\n----------------------------------------------------------------------------------------\n")

# Extract vocabulary(distinct words) from training, validation, and test data
train_vocab = set(word for review in train_df['content'] for word in review.split())
val_vocab = set(word for review in val_df['content'] for word in review.split())
test_vocab = set(word for review in test_df['content'] for word in review.split())

# Build the GloVe dictionary for the reviews
glove_dict = create_glove_dict(train_vocab, wv, "training")
glove_dict.update(create_glove_dict(val_vocab, wv, "validation"))
glove_dict.update(create_glove_dict(test_vocab, wv, "test"))

# Convert reviews to glove embeddings
train_review_gloves = get_w2GloVe(train_df['content'], glove_dict, "training")
val_review_gloves = get_w2GloVe(val_df['content'], glove_dict, "validation")
test_review_gloves = get_w2GloVe(test_df['content'], glove_dict, "test")

print(f"----------------------------------------------------------------------------------------\nProcessing Rationales\n----------------------------------------------------------------------------------------\n")

#Extract vocabulary(distinct words) from training, validation, and test data for the rationales
train_rationale_vocab = set(word for rationale in train_rationales for word in rationale.split())
val_rationale_vocab = set(word for rationale in val_rationales for word in rationale.split())
test_rationale_vocab = set(word for rationale in test_rationales for word in rationale.split())

# Build the GloVe dictionary for the rationales
dict_rat = create_glove_dict(train_rationale_vocab, wv, "training")
dict_rat.update(create_glove_dict(val_rationale_vocab, wv, "validation"))
dict_rat.update(create_glove_dict(test_rationale_vocab, wv, "test"))

# Convert rationales to glove embeddings
train_rationale_gloves = get_w2GloVe(train_rationales, glove_dict, "training", rationale=True)
val_rationale_gloves = get_w2GloVe(val_rationales, glove_dict, "validation", rationale=True)
test_rationale_gloves = get_w2GloVe(test_rationales, glove_dict, "test", rationale=True)

----------------------------------------------------------------------------------------
Processing Reviews
----------------------------------------------------------------------------------------



Retrieving training GloVe Word Embeddings:   0%|          | 0/1600 [00:00<?, ?it/s]<ipython-input-13-cee810f9ddc6>:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  glove_reviews.append(torch.tensor(embeddings, dtype=torch.float))
Retrieving test GloVe Word Embeddings: 100%|██████████| 199/199 [00:01<00:00, 135.87it/s]


----------------------------------------------------------------------------------------
Processing Rationales
----------------------------------------------------------------------------------------



Retrieving test GloVe Word Embeddings: 100%|██████████| 199/199 [00:00<00:00, 4871.29it/s]


Save the GloVe embeddings to local files for faster Access.

In [15]:
from torch import Tensor
from typing import Dict, List, Optional, Tuple, Union
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

with open("train_reviews.pkl", "wb") as f:
    pickle.dump(train_review_gloves, f)

with open("val_reviews.pkl", "wb") as f:
    pickle.dump(val_review_gloves, f)

with open("test_reviews.pkl", "wb") as f:
    pickle.dump(test_review_gloves, f)

with open("train_rationales.pkl", "wb") as f:
    pickle.dump(train_rationale_gloves, f)

with open("val_rationales.pkl", "wb") as f:
    pickle.dump(val_rationale_gloves, f)

with open("test_rationales.pkl", "wb") as f:
    pickle.dump(test_rationale_gloves, f)


# Extract the GloVe embeddings created above and a create a copy before batching them.

In [16]:
with open("train_reviews.pkl", "rb") as f:
    train_in = pickle.load(f)

with open("train_rationales.pkl", "rb") as f:
    train_ev = pickle.load(f)

with open("val_reviews.pkl", "rb") as f:
    val_in = pickle.load(f)

with open("val_rationales.pkl", "rb") as f:
    val_ev = pickle.load(f)

with open("test_reviews.pkl", "rb") as f:
    test_in = pickle.load(f)

with open("test_rationales.pkl", "rb") as f:
    test_ev = pickle.load(f)

Convert the training, validation, and test data(GloVe representations) including the rationales to batches using DataLoader

In [17]:
train_inputs = TensorDataset(train_in, train_ev, train_classes)
val_inputs = TensorDataset(val_in, val_ev, val_classes)
test_inputs = TensorDataset(test_in, test_ev, test_classes)

train_loader = DataLoader(train_inputs, batch_size=32, shuffle=True)
val_loader = DataLoader(val_inputs, batch_size=25, shuffle=False)
test_loader = DataLoader(test_inputs, batch_size=25, shuffle=False, drop_last=False)

# Convolutional Neural Network Model

In [18]:
from typing import List, Optional, Tuple, Union
from torch import Tensor
import torch.nn as nn

class LIME_CNN(nn.Module):
    def __init__(
        self,
        embed_dim: int,
        cnn_config: List[Dict],
    ) -> None:

        super(LIME_CNN, self).__init__()

        self.in_channels = embed_dim

        self.config = cnn_config

        self.cnn_layers = nn.ModuleList()

        for index, config in enumerate(cnn_config):

            conv_layer = nn.Conv1d(
                in_channels=self.in_channels,
                out_channels=config['out_channels'],
                kernel_size=config['kernel_size'],
                stride=config['stride'],
                padding=config['padding'],
                bias=config['bias'],
            )
            self.cnn_layers.append(conv_layer)

            if index == 0 or index == len(cnn_config) - 1:

                maxPool_layer = nn.MaxPool1d(
                    kernel_size=config['kernel_size'],
                    stride=config['stride'],
                    padding=config['padding'],
                )
                self.cnn_layers.append(maxPool_layer)

        self.relu = nn.ReLU()

    def forward(
        self,
        X: Tensor,
    ) -> Tensor:

        X = X.permute(0, 2, 1)
        for layer in self.cnn_layers:

            conv_out = self.relu(layer(X)) if isinstance(layer, nn.Conv1d) else layer(X)

        X = X.mean(dim=-1)

        self.fc_layer = nn.Linear(X.shape[1], 1)

        y_hat = self.fc_layer(X)

        y_hat = torch.sigmoid(y_hat).squeeze(-1)

        return y_hat


# Training Loop

In [36]:
import json
# Load the model configuration from a JSON file
with open("model_config.json", "r") as f:
    loaded_config = json.load(f)

def train_model(model, train_set, val_set, n_epochs, lr):

    criterion = torch.nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_loss = float('inf') # initialize the best loss the model can achieve to infinity
    patience = 0 # initialize the patience for early stopping if validation loss plateaus
    loss_list = []
    for epoch in range(n_epochs):

        model.train()
        train_loss = 0.0
        avg_loss = 0.0
        predictions = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            inputs, rationales, labels = batch
            pred = model(inputs)
            predictions.append(pred)
            loss = criterion(pred, labels)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        loss_list.append(train_loss)
        print(f"Training Loss: {train_loss:.4f}")

        model.eval()
        val_loss = 0.0
        tp = 0
        num_labels = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, rationales, labels = batch
                pred = model(inputs)
                loss = criterion(pred, labels)
                val_loss += loss.item()

                y_hats = (pred > 0.5).float()
                tp += torch.sum(y_hats == labels).item()
                num_labels += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = tp / num_labels
        print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}\n")

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), "best_model_state.pt")

    return predictions, train_loss, loss_list

model = LIME_CNN(embed_dim=50, cnn_config=loaded_config)

preds, train_loss, loss_list = train_model(
                                model=model,
                                train_set=train_loader,
                                val_set=val_loader,
                                n_epochs=5,
                                lr=0.01,
                            )

Epoch 1/5: 100%|██████████| 50/50 [00:07<00:00,  6.50it/s]


Training Loss: 0.6938
Validation Loss: 0.6916, Accuracy: 0.3500



Epoch 2/5: 100%|██████████| 50/50 [00:07<00:00,  6.31it/s]


Training Loss: 0.6946
Validation Loss: 0.6850, Accuracy: 0.6250



Epoch 3/5: 100%|██████████| 50/50 [00:07<00:00,  6.42it/s]


Training Loss: 0.6958
Validation Loss: 0.6772, Accuracy: 0.6950



Epoch 4/5: 100%|██████████| 50/50 [00:07<00:00,  6.78it/s]


Training Loss: 0.6951
Validation Loss: 0.6945, Accuracy: 0.6100



Epoch 5/5: 100%|██████████| 50/50 [00:07<00:00,  6.33it/s]


Training Loss: 0.6945
Validation Loss: 0.6775, Accuracy: 0.6100



In [45]:
from plotly import express as px

loss_list = pd.DataFrame(loss_list)
loss_list.columns = ['loss']
loss_list.head()

fig = px.scatter(loss_list, x=loss_list.index, y='loss')
fig.show()
